In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from sklearn.cluster import KMeans 
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline 

In [2]:
CLIENT_ID = 'VJT3DPD440S05DGHYYPNHGHQ4NPSHO0WX2P5F34EMFDPF3JC' 
CLIENT_SECRET = 'TAE25NKJQD35LK4JJLQU5HZ1W55WMYKU0OQKHZYQUEXF3XL4' 
ACCESS_TOKEN = 'Q0O20AEPKFGC5YLKIDZJIJMCF3EQ4B3SOKBEE1OJGISUPZQ0'
VERSION = '20180604'
LIMIT = 30

#Tokens removed from upload for privacy

# Business Problem
A person wants to start a sandwich shop in Toronto. This person is very experienced at starting sandwich shops and has the business model, standard operating procedures, and financial planning in place. This person believes that they can offer an enticing menu while operating more efficiently than the competition. However, this will be the first of the person's shops in Toronto so they want to hit the market with high potency: this means that they want to be the best shop in the first area they open in so that sentiment toward their brand builds positively in a very quick manner.

### Thus, our business problem is as follows:
- Identify areas with relatively few sandwich shops
- Identify areas with relatively low ratings on restaurants

We will use the intersection of those areas to determine candidate neighborhoods to open up shop in. To break ties, if there are any, we will simulate personal whim and taste using a random choice in the end.

# Data
In addition to the neighborhood postal codes and GPS coordinates for Toronto that were used in previous parts of this project, we will pull data from the Foursquare API pertaining to the venues in each cluster of neighborhoods that were chosen previously. Clusters will be chosen so that deliveries within each cluster would take roughly 5 minutes at most.

Data will be filtered and aggregated to determine:
- How many restaurants there are
    - more so, how many are sandwich shops
- What the average rating for a restaurant is in a cluster.

We will choose the areas with the lowest numbers on those metrics so that we can shock the market with excellence.

# Beginning of Data Prep
First, I will cluster neighborhoods that are close together. Then I will query the venues that serve the centroids of each cluster and go from there.

In [3]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [4]:
geodata = pd.read_csv("Geospatial_Coordinates.csv")
geodata.rename(columns={"Postal Code":"PostalCode"},inplace=True)
geodata.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
df = df.merge(geodata,on="PostalCode")
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [6]:
X = df[["Latitude","Longitude"]]
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ibm_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
num_clusters = 20

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(X)
labels = k_means.labels_

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=9.5)

x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to map
for lat, lng, borough, neighborhood, cluster in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood'],labels):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 

map_toronto

In [12]:
df["ClusterLabel"] = labels

In [13]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,ClusterLabel
0,M3A,North York,Parkwoods,43.753259,-79.329656,5
1,M4A,North York,Victoria Village,43.725882,-79.315572,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,7
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,2


In [16]:
k_means.cluster_centers_

array([[ 43.70968378, -79.30711863],
       [ 43.741392  , -79.4942899 ],
       [ 43.65376691, -79.38182756],
       [ 43.80911537, -79.18682887],
       [ 43.64118192, -79.55191292],
       [ 43.7437839 , -79.32965415],
       [ 43.70874513, -79.39681095],
       [ 43.690505  , -79.46523107],
       [ 43.73659725, -79.2535714 ],
       [ 43.734156  , -79.5828182 ],
       [ 43.76979918, -79.4203107 ],
       [ 43.6351939 , -79.50244454],
       [ 43.66268167, -79.42899413],
       [ 43.7976538 , -79.29232435],
       [ 43.67692348, -79.3362865 ],
       [ 43.7816793 , -79.36767413],
       [ 43.70421658, -79.5343494 ],
       [ 43.6369656 , -79.615819  ],
       [ 43.6845072 , -79.27893965],
       [ 43.76923357, -79.215035  ]])

In [19]:
df["CentroidLat"] = df.apply(lambda row: k_means.cluster_centers_[row.ClusterLabel][0] ,axis=1)
df["CentroidLon"] = df.apply(lambda row: k_means.cluster_centers_[row.ClusterLabel][1] ,axis=1)

In [20]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,ClusterLabel,CentroidLat,CentroidLon
0,M3A,North York,Parkwoods,43.753259,-79.329656,5,43.743784,-79.329654
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,43.709684,-79.307119
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,43.653767,-79.381828
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,7,43.690505,-79.465231
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,2,43.653767,-79.381828


In [31]:
search_query = 'Deli'
radius = 3000

DFs = []

for coordinates in k_means.cluster_centers_:
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, coordinates[0], coordinates[1],ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
    result = requests.get(url).json()
    
    try:
        DFs.append(pd.json_normalize(result['response']['venues']))
    except:
        DFs.append(None)

In [39]:
for each in DFs:
    display(each.head())
    print(each.shape)

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,4bc5dfbf4a9aa593a00b077b,Dr. Deli & The Salad Queen,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",v-1623927873,False,1881 Steeles Ave W,Dufferin St,43.726865,-79.292786,"[{'label': 'display', 'lat': 43.726865, 'lng':...",2233,M3H 5T7,CA,Toronto,ON,Canada,"[1881 Steeles Ave W (Dufferin St), Toronto ON ...",NaN,NaN
1,4dbf46c41e72dd48b1ef56ef,Oak Park Deli,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1623927873,False,NaN,NaN,43.693877,-79.310264,"[{'label': 'display', 'lat': 43.69387680661727...",1777,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN
2,5954162804d1ae42efb9b367,Druxy's Famous Deli,"[{'id': '56aa371be4b08b9a8d573550', 'name': 'F...",v-1623927873,False,1200 Eglinton Avenue East,NaN,43.722408,-79.334498,"[{'label': 'display', 'lat': 43.7224083, 'lng'...",2618,M3C 1H9,CA,Toronto,ON,Canada,"[1200 Eglinton Avenue East, Toronto ON M3C 1H9...",NaN,NaN
3,4f5a4771e4b0357e5d6b1667,Jody's Deli,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",v-1623927873,False,777 Warden Avenue,NaN,43.724536,-79.286347,"[{'label': 'display', 'lat': 43.72453554078253...",2350,NaN,CA,Toronto,ON,Canada,"[777 Warden Avenue, Toronto ON, Canada]",NaN,NaN
4,4b92e7e9f964a520f32534e3,Smiley's Restaurant and Deli,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",v-1623927873,False,2543 Av Danforth Ave,Main and danforth,43.688106,-79.302148,"[{'label': 'display', 'lat': 43.68810648196892...",2435,NaN,CA,Toronto,ON,Canada,"[2543 Av Danforth Ave (Main and danforth), Tor...",NaN,NaN


(22, 19)


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.postalCode,location.city,location.state,location.address,location.crossStreet
0,5216466a11d2b508c44da374,Deli Break,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1623927873,False,43.755304,-79.466502,"[{'label': 'display', 'lat': 43.75530383603603...",2718,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN
1,5a9b0ba9da5e5656bc780865,Santo Fine Meat & Deli,"[{'id': '50be8ee891d4fa8dcc7199a7', 'name': 'M...",v-1623927873,False,43.722730,-79.498080,"[{'label': 'display', 'lat': 43.72273, 'lng': ...",2099,CA,Canada,"[Toronto ON M3M 1H7, Canada]",M3M 1H7,Toronto,ON,NaN,NaN
2,53989d0e498e7e5f942bbe19,Deco Deli,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",v-1623927873,False,43.765030,-79.483467,"[{'label': 'display', 'lat': 43.76502990722656...",2771,CA,Canada,"[1183 Finch Ave. W, North York ON, Canada]",NaN,North York,ON,1183 Finch Ave. W,NaN
3,4fede8cfe4b00e3bf7fde289,Narin Deli & Pastei,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",v-1623927873,False,43.730681,-79.466486,"[{'label': 'display', 'lat': 43.73068085997674...",2534,CA,Canada,"[Toronto ON, Canada]",NaN,Toronto,ON,NaN,NaN
4,4e5d7f0052b19f7d4741cffd,Loconte Fine Meats & Deli,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",v-1623927873,False,43.729884,-79.463750,"[{'label': 'display', 'lat': 43.7298836, 'lng'...",2770,CA,Canada,"[736 Wilson (Lady York Ave), North York ON M3K...",M3K 1E2,North York,ON,736 Wilson,Lady York Ave


(11, 17)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood
0,4b8c0d41f964a520e5ba32e3,Marcello's Market and Deli,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",v-1623927873,False,393 University Ave.,University Centre,43.653761,-79.388098,"[{'label': 'display', 'lat': 43.65376090619767...",504,M5G 1E6,CA,Toronto,ON,Canada,"[393 University Ave. (University Centre), Toro...",NaN
1,4e9cc51949017c00c846f45b,Waterfront Deli,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",v-1623927873,False,NaN,NaN,43.641491,-79.375872,"[{'label': 'display', 'lat': 43.64149145036152...",1448,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN
2,4af5b3a5f964a5208dfb21e3,New Yorker Deli,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",v-1623927873,False,1140 Bay Street,at Charles St,43.668533,-79.388955,"[{'label': 'display', 'lat': 43.66853282691770...",1741,M5S 2B4,CA,Toronto,ON,Canada,"[1140 Bay Street (at Charles St), Toronto ON M...",NaN
3,4b1951c1f964a5202edb23e3,King Deli Cafe,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",v-1623927873,False,465 King Street E,just East of Parliament St,43.653906,-79.361147,"[{'label': 'display', 'lat': 43.65390613999999...",1665,NaN,CA,Toronto,ON,Canada,[465 King Street E (just East of Parliament St...,NaN
4,4c47d78c1ddec92817ca9e32,Micheal's Restaurant and Deli,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1623927873,False,566 Queen St W,NaN,43.647289,-79.403493,"[{'label': 'display', 'lat': 43.64728932229506...",1888,NaN,CA,Toronto,ON,Canada,"[566 Queen St W, Toronto ON, Canada]",NaN


...

(16, 17)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,556b585d498e40bf371e1f7d,Wave Deli,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",v-1623927876,False,1036 Kingston Rd,Victoria Park Ave & Kingston Rd,43.681024,-79.284569,"[{'label': 'display', 'lat': 43.68102443523008...",596,M4E 1T4,CA,Toronto,ON,Canada,[1036 Kingston Rd (Victoria Park Ave & Kingsto...
1,576c03d4498e3bc43cb6ec54,M&J's Eastside Deli,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",v-1623927876,False,NaN,NaN,43.687432,-79.271352,"[{'label': 'display', 'lat': 43.687432, 'lng':...",692,M1N 1R4,CA,Toronto,ON,Canada,"[Toronto ON M1N 1R4, Canada]"
2,4cbdd779d78f468834eccf73,Grumble's deli-cafe,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",v-1623927876,False,290 Main St.,at Danforth Ave.,43.687581,-79.301735,"[{'label': 'display', 'lat': 43.68758138785994...",1866,M4C,CA,Toronto,ON,Canada,"[290 Main St. (at Danforth Ave.), Toronto ON M..."
3,4b92e7e9f964a520f32534e3,Smiley's Restaurant and Deli,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",v-1623927876,False,2543 Av Danforth Ave,Main and danforth,43.688106,-79.302148,"[{'label': 'display', 'lat': 43.68810648196892...",1910,NaN,CA,Toronto,ON,Canada,"[2543 Av Danforth Ave (Main and danforth), Tor..."
4,4dbf46c41e72dd48b1ef56ef,Oak Park Deli,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1623927877,False,NaN,NaN,43.693877,-79.310264,"[{'label': 'display', 'lat': 43.69387680661727...",2728,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]"


(14, 17)


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.postalCode,location.city,location.state,location.neighborhood,location.address,venuePage.id
0,576443c4498ee4a134fe9228,Polka European Deli Inc.,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",v-1623927877,False,43.760137,-79.227173,"[{'label': 'display', 'lat': 43.760137, 'lng':...",1406,CA,Canada,[Canada],NaN,NaN,NaN,NaN,NaN,NaN
1,5af738c0947c05003935ef99,Chick·n·delish,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1623927877,False,43.773660,-79.230400,"[{'label': 'display', 'lat': 43.77366, 'lng': ...",1329,CA,Canada,"[Toronto ON M1H 3K1, Canada]",M1H 3K1,Toronto,ON,NaN,NaN,NaN
2,50bf6d18e4b01c89d3763d51,Birthing And Delivery,"[{'id': '56aa371be4b08b9a8d5734ff', 'name': 'M...",v-1623927877,False,43.780685,-79.205927,"[{'label': 'display', 'lat': 43.780685, 'lng':...",1470,CA,Canada,[Canada],NaN,NaN,NaN,Morningside,NaN,NaN
3,5f7390942142964ba8d2fdfa,Walmart Grocery Pickup & Delivery,"[{'id': '52f2ab2ebcbc57f1066b8b46', 'name': 'S...",v-1623927877,False,43.797790,-79.200879,"[{'label': 'display', 'lat': 43.79779047136888...",3376,CA,Canada,"[799 Milner Ave, Scarborough ON M1B 3C3, Canada]",M1B 3C3,Scarborough,ON,NaN,799 Milner Ave,597157345


(4, 18)


In [50]:
dfLengths = []

for each in DFs:
    dfLengths.append(each.shape[0])
    
numCompetitors = np.array(dfLengths)

In [46]:
centerLats = []
centerLons = []
clusterNums = []

i = 0
centers = k_means.cluster_centers_
while i < len(centers):
    clusterNums.append(i)
    centerLats.append(centers[i][0])
    centerLons.append(centers[i][1])
    
    i += 1
        

In [48]:
centerLats = np.array(centerLats)
centerLons = np.array(centerLons)
clusterNums = np.array(clusterNums)

In [47]:
centroidDetails = pd.DataFrame()

In [51]:
centroidDetails["ClusterNumber"] = clusterNums
centroidDetails["Latitude"] = centerLats
centroidDetails["Longitude"] = centerLons
centroidDetails["CompetitorCount"] = numCompetitors

In [52]:
centroidDetails.head()

,ClusterNumber,Latitude,Longitude,CompetitorCount
0,0,43.709684,-79.307119,22
1,1,43.741392,-79.494290,11
2,2,43.653767,-79.381828,30
3,3,43.809115,-79.186829,1
4,4,43.641182,-79.551913,13


In [56]:
avgRatings = []

for DF in DFs:
    ratings = []
    for venueId in DF["id"]:
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venueId, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
        result = requests.get(url).json()
        rating = 0
        try:
            rating = result['response']['venue']['rating']
            ratings.append(rating)
        except:
            pass
        
        
    if len(ratings) == 0:
        avgRatings.append(0)
    else:
        avgRatings.append(sum(ratings)/len(ratings))
    
avgRatings

[6.9,
 0,
 6.640000000000001,
 0,
 8.4,
 7.133333333333333,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [57]:
centroidDetails["AverageRating"] = np.array(avgRatings)

In [59]:
centroidDetails

,ClusterNumber,Latitude,Longitude,CompetitorCount,AverageRating
0,0,43.709684,-79.307119,22,6.900000
1,1,43.741392,-79.494290,11,0.000000
2,2,43.653767,-79.381828,30,6.640000
3,3,43.809115,-79.186829,1,0.000000
4,4,43.641182,-79.551913,13,8.400000
5,5,43.743784,-79.329654,26,7.133333
6,6,43.708745,-79.396811,30,0.000000
7,7,43.690505,-79.465231,21,0.000000
8,8,43.736597,-79.253571,7,0.000000
9,9,43.734156,-79.582818,7,0.000000


In [64]:
lat = 43.769234
lng = -79.215035
#cluster 19

map2 = folium.Map(location=[lat,lng], zoom_start=9.5)
folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup="here",
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.7,
        parse_html=False).add_to(map2)
map2